# Demo for reading and summarizing research articles
Since studies are almost always in PDF format, we need to find a way to read them and get meaningful information out of them. For this purpose, I will be using PyPDF2 to extract the plaintext, and openAI's GPT language model to make sense of the output.


In [81]:
import os
os.environ['CRYPTOGRAPHY_DONT_BUILD_RUST'] = "1"
%pip install PyPDF2
%pip install openai
%pip install pdfplumber

^C
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\tzhou\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\tzhou\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\tzhou\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


## Text Extraction
First, upload a paper in the local directory. Then, use PyPDF2 to iterate and read each page and store each page in a list

In [99]:
from PyPDF2 import PdfReader

#this visitor function helps with cutting off unnecessary header/footer text
def visitor_body(text, cm, tm, font_dict, font_size):
    y = tm[5]
    if y > 10 and y < 780:
        parts.append(text.lower())
        
reader = PdfReader("paper1.pdf")
number_of_pages = len(reader.pages)
pageList = [""] * number_of_pages
for i in range(number_of_pages):
    parts = []
    reader.pages[i].extract_text(visitor_text=visitor_body)
    text_body = "".join(parts)
    text_body = text_body.replace(' ', '')
    pageList[i] = text_body


## Organizing data by segment

To better organize the papers, we can try to parse each section of the paper. Every paper has a methods section, results section, and discussion section. Once we have the sections parsed, we have a better chance of the language model understanding the data we present.

In [103]:
fullText = ""
for page in pageList:
    # print(page)
    fullText += page

methodsStartInd = fullText.find('andmethods\n') + 10
resultStartInd = fullText[methodsStartInd: ].find('results\n') + methodsStartInd + 7
discussionStartInd = fullText[resultStartInd: ].find('discussion\n') + resultStartInd + 10
referencesStartInd = fullText[discussionStartInd: ].find('references') + discussionStartInd + 10
methods = fullText[methodsStartInd: resultStartInd]
results = fullText[resultStartInd: discussionStartInd]
discussion = fullText[discussionStartInd: referencesStartInd]
# print(fullText)
print(discussion)
# print(discussionStartInd, referencesStartInd)


riskofovariancarcinomaforwomenwhoeverusedtalcumresultsfromexperimentalandepidemiologicstudies
powderregularly,bycasehistologyconductedthusfarindicateapossibleassociationbe-
tweentalcexposureandovariancarcinoma.histologicno.(%)whoadjusteda
evidence®rstindicatedthatcontaminantssuchtalctotalno.usedtalcum
histologictypeofcasespowderor(95%ci)maybecomeembeddedinovariantumors.8,24experi-
mentalstudieshaveshownthatexternaltalcexposure
invasive367166(45.2)1.513(1.13±2.02)mayeventuallyreachtheovaries.hendersonetal.10
borderline8332(38.6)1.237(0.76±2.02)
demonstratedthattalcwaspresentinovariesafterserous254109(42.9)1.336(0.96±1.85)mucinous8035(43.8)1.585(0.97±2.58)
depositionofatalcsuspensioninthevaginaandcervi-
endometrioid7436(48.6)1.671(1.00±2.79)calosinrats.similarly,egliandnewton9revealedin
humanstudiesthatinertcarbonparticlesdeposited
or:oddsratio;ci:con®denceinterval.inthevaginacanlaterberecoveredinthefallopianaadjustedasintable1.
tubes.althoughthesestudiesdemonstratedapossible
routeofexp

## Extracting features

starting with: “conclusion”, “no. of subjects”, “relative risk”, “length of follow”

In [108]:
import openai

#first is the easiest, extract keywords that the paper volunteers on the first page, if there are any
i = 0
while(pageList[i] == ""):
    i+= 1

firstPage = pageList[i]
ind = firstPage.find('keywords:') + 9
if ind != -1:
    newLine = firstPage[ind: ].find('\n') + ind
    keywords = firstPage[ind: newLine]

#next, we can prompt GPT for conclusion and study size data
print(methods)
print(results)
print(discussion)

openai.api_key = ""

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": f"You are a helpful research assistant. This is the results section of an academic paper {results}"},
        {"role": "user", "content": "Answer these questions with one sentence or less. What is the conclusion? What is the study size? The study duration? How strong was the evidence?"},
        ],
    )
print(response["choices"][0]["message"]["content"])




studymethodshavebeenreportedindetailelse-offull-termpregnancies;averagedurationof
breastfeedingperpregnancy;andeverhavinghadawhere22andwillbesummarizedhere.ourstudypopu-
lationconsistedofwomenbetweentheagesof35andtuballigation,ahysterectomy,oramotherorsister
withovarianorbreastcarcinoma.79yearsresidinginthehighlypopulatedareasur-
roundingthewesternendoflakeontario,canada.caseswerewomenwhohadhistologicallycon®rmed
results

table1showsthedescriptivecharacteristicsoftheprimary,invasiveorborderlineepithelialovariantu-
mors®rstdiagnosedbetweennovember1,1989,and450ovariancarcinomacasesandthe564population
controls.ageatinterviewwasusedasamatchingvari-october31,1992.ofthe631womenidenti®edas
cases,450(71.3%)wereinterviewed.fifty-®ve(8.7%)able.asobservedinmanyreports,20,22,23controlshad,
onaverage,agreaternumberoffull-termpregnancies.haddied,butproxyinterviewswerenotconducted;29
(4.6%)hadphysicianswhorefusedconsent;30(4.8%)ahigherpercentageofcontrolshadhadatuballiga-
tionorahysterectomy,whereas

In [106]:
print(response["choices"][0]["message"]["content"])

Conclusion: Talc exposure is associated with an increased risk of ovarian cancer. 

Study size: 631 women with ovarian cancer and 564 population controls. 

Study duration: November 1, 1989 to October 31, 1992. 

Strength of evidence: The study found a statistically significant association between talc use and risk of ovarian cancer.
